# Capacitated Vehicle Routing Problem

First, we import some useful python packages for solving the capacitated vehicle routing problem.

In [2]:
from IPython.display import display, HTML
import openrouteservice
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# Setting up the problem

For the capacitated vehicle routing problem we will use the dataset from "Augerat, et al. Set A" obtained at https://www.coin-or.org/SYMPHONY/branchandcut/VRP/data/index.htm.old#A. The dataset contains 32 nodes (including 1 depot) with their demands and 5 vehicles with a capacity of 100 each. We first import the dataset and extract the relevant information from the dataset. The following table shows the extracted data. The x-coordinates and y-coordinates represent the position of the nodes. The column "demand" shows the "demanded order" that needs to be delivered to the respective node. The node with zero demand is the depot (first node).  

In [3]:
def str_to_num(lst_2d, num_type='int'):
    for i, item in enumerate(lst_2d):
        for j, num in enumerate(item):
            if num_type == 'int':
                lst_2d[i][j] = int(num)
            elif num_type == 'float':
                lst_2d[i][j] = float(num)
    return lst_2d

def read_file(file_path): # "A-n32-k5.vrp"
    decider = 0 
    nodes = []
    demands = []
    with open(file_path, 'r') as file:
        for line in file:
            if line == 'NODE_COORD_SECTION \n':
                decider = 1
            elif line == 'DEMAND_SECTION \n':
                decider = 2
            elif line == 'DEPOT_SECTION \n':
                decider = 0
        
            if decider == 0:
                continue
            elif decider == 1:
                nodes.append(line.lstrip().rstrip().split(' '))
            elif decider == 2:
                demands.append(line.lstrip().rstrip().split(' '))
    file.close()
    nodes = np.delete(np.array(str_to_num(nodes[1:])), 0, 1)
    demands = np.delete(np.array(str_to_num(demands[1:])), 0, 1)
    data = np.concatenate((nodes, demands), axis=1)
    return data

data = read_file("A-n32-k5.vrp")
df_data = pd.DataFrame(data, columns=['x-coordinate', 'y-coordinate', 'demand'])
df_data.columns.name = 'node number'
display(df_data)

node number,x-coordinate,y-coordinate,demand
0,82,76,0
1,96,44,19
2,50,5,21
3,49,8,6
4,13,7,19
5,29,89,7
6,58,30,12
7,84,39,16
8,14,24,6
9,2,39,16


To give the rather abtract problem a real world application, we map the nodes to locations in the german city Heidelberg. First, we need to calculate the borders of Heidelberg. For this endeavor, we use the open street map and a python script (http://polygons.openstreetmap.fr/index.py) to compute the polygon representing the borders of Heidelberg. The generated file needs to be read into our jupyter notebook and prepared in such a way that it is useful for further processing steps. Furthermore, we compute the centroid of the polygon and plot it with the folium package to visualize the polygon and double-check, if everything went well. 

In [5]:
def read_file_polygon(file_path): # 'HD_polygon.txt'
    with open(file_path, 'r') as file:
        polygon = []
        for line in file:
            polygon.append(line.lstrip().rstrip().split('\t'))
        polygon = np.array(str_to_num(polygon, 'float'))
    file.close()
    return polygon

def get_geometry_polygon(polygon):
    polygon_str = ""
    for item in polygon:
        polygon_str += str(item)[2:-1] + ", " 
    polygon_str = 'Polygon ((' + polygon_str[:-2] + '))'    
    poly_geom = wkt.loads(polygon_str) # load geometry from Polygon string
    poly_coords = list(poly_geom.exterior.coords) # get coords from exterior ring
    poly_coords = [(y,x) for x,y in poly_coords] # swap (x,y) to (y,x). Really leaflet?!
    poly_centroid = poly_geom.centroid 
    return poly_coords, poly_centroid

polygon = read_file_polygon('HD_polygon.txt')
print(polygon)

[[  8.5734302  49.4157627]
 [  8.573288   49.416518 ]
 [  8.5732227  49.4169018]
 ..., 
 [  8.5739798  49.4147232]
 [  8.573622   49.4157339]
 [  8.5734302  49.4157627]]
